<a href="https://colab.research.google.com/github/DASIREDDYCHAITANYA/Practice_hub-GEN-AI/blob/main/(Langchian)Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install wikipedia
!pip install langchain_community
!pip install langchain_huggingface
!pip install langchain_chroma
!pip install pypdf

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia.run("What is the capital of France?")

"Page: Closed-ended question\nSummary: A closed-ended question is any question for which a researcher provides research participants with options from which to choose a response. Closed-ended questions are sometimes phrased as a statement that requires a response.\nA closed-ended question contrasts with an open-ended question, which cannot easily be answered with specific information.\n\n\n\nPage: France\nSummary: France, officially the French Republic, is a country primarily located in Western Europe. Its overseas regions and territories include French Guiana in South America, Saint Pierre and Miquelon in the North Atlantic, the French West Indies, and many islands in Oceania and the Indian Ocean, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north; Germany to the northeast; Switzerland to the east; Italy and Monaco to the southeast; Andorra and Spain to the south; and a maritime 

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/MS_Dhoni")

In [ ]:
doc=loader.load()
doc

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/MS_Dhoni', 'title': 'MS Dhoni - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nMS Dhoni - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
spliter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
data=spliter.split_documents(doc)
data[:100]

In [ ]:
vectordb=Chroma.from_documents(data,embeddings)
retriver=vectordb.as_retriever()
retriver

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool

In [ ]:
pdf=PyPDFLoader("/content/attention.pdf").load()
pdf=spliter.split_documents(pdf)

In [ ]:
vector1=Chroma.from_documents(pdf,embeddings)


In [ ]:
retrivel_tool=create_retriever_tool(vector1.as_retriever(),'ATTENTION',"Search for information about attention model. For any questions about attention model, you must use this tool!")

In [ ]:
retrivel_tool1=create_retriever_tool(retriver,"MS DHONI","Search for information about MS Dhoni. For any questions about MS Dhoni, you must use this tool!")

In [ ]:
retrivel_tool1.name

'MS DHONI'

In [ ]:
tools=[wikipedia,retrivel_tool1,retrivel_tool]

In [ ]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='MS DHONI', description='Search for information about MS Dhoni. For any questions about MS Dhoni, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7d3d8e4def20>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d3d962c7990>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7d3d8e521440>, retriever=VectorStoreRetriever(tags=['

In [ ]:

import getpass
import os

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your token: ")

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

model_id = "google/flan-t5-large"  # 1024 token limit
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Move the model to the CPU
model.to('cpu')

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=700, truncation=True)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain import hub

In [ ]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [ ]:
from langchain.agents import create_react_agent
from langchain import hub

# Pull a suitable prompt for the zero_shot_react_description agent
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor
agent_Executor=AgentExecutor(agent=agent,tools=tools,verbose=False, handle_parsing_errors=True)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        verbose=False,
        handle_parsing_errors=True,
        max_iterations=4000,
        max_execution_time=60000,
        return_intermediate_steps=True
    )

In [ ]:
agent_Executor.invoke({"input":"Tell me about dhoni"})

{'input': 'Tell me about dhoni',
 'output': 'Agent stopped due to iteration limit or time limit.'}